### Seminar DL

Here is the code we use to get a good grade.

In [117]:
# Import libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
import sklearn as sk
#from tf.keras import backend as k
from tensorflow.python.keras import layers
from sklearn.model_selection import train_test_split
#from tensorflow.python.keras.models import Sequential


In [118]:
print(tf.__version__)
print(tf.keras.__version__)

2.0.0-beta1
2.2.4-tf


In [119]:
# Check working directory
import os
print(os.getcwd())

C:\Users\cjb86


#### Preprocessing

#### ATP model

In [120]:
# Load prepocessed data
atp_data = pd.read_csv('atp_dataedit1.csv')

In [121]:
atp_data.columns


Index(['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface',
       'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'Wsets',
       'Lsets', 'Comment', 'PSW', 'PSL', 'B365W', 'B365L', 'elo_winner',
       'elo_loser', 'proba_elo'],
      dtype='object')

In [122]:

atp_data['year'] = atp_data.Date.astype(str).str[-10:-6]
atp_data['date'] = pd.to_datetime(atp_data['Date'], format = '%Y%m%d', errors = 'coerce')

In [123]:
#replicate dataset and swap variable to create a stack
atp_data2 = atp_data.copy()
atp_data2.rename(columns = {'WRank' : 'LRank',
                            'PSW' : 'PSL',
                            'B365W' : 'B365L',
                            'elo_winner' : 'elo_loser',
                            'LRank' : 'WRank', 
                            'PSL':'PSW',
                            'B365L':'B365W',
                            'elo_loser' : 'elo_winner'},
                inplace = True)
print(atp_data[['WRank', 'LRank', 'PSW', 'PSL', 'B365W', 'B365L', 'elo_winner','elo_loser']].tail(3))
print(atp_data2[['WRank', 'LRank', 'PSW', 'PSL', 'B365W', 'B365L', 'elo_winner','elo_loser']].tail(3))

atp_data['won'] = 1 
atp_data2['won'] = 0 
#stack
atp_stack = atp_data.append(atp_data2, ignore_index = True, sort = True)

       WRank  LRank   PSW   PSL  B365W  B365L   elo_winner    elo_loser
17653      6      8  1.37  3.48   1.33   3.25  1956.268895  1903.306531
17654    169     26  2.24  1.71   2.20   1.61  1723.326255  1793.794657
17655    209     44  3.14  1.41   2.75   1.40  1494.041611  1697.370640
       WRank  LRank   PSW   PSL  B365W  B365L   elo_winner    elo_loser
17653      8      6  3.48  1.37   3.25   1.33  1903.306531  1956.268895
17654     26    169  1.71  2.24   1.61   2.20  1793.794657  1723.326255
17655     44    209  1.41  3.14   1.40   2.75  1697.370640  1494.041611


In [124]:
#drop unused variables
data = atp_data.drop(['B365W','B365L','PSL','PSW','Comment','ATP','Date','Winner','Loser'],axis = 1)

dummy_fields = ['Surface', 'Tournament', 'Round', 'Court', 'Series']
for each in dummy_fields:
    dummies = pd.get_dummies(data[each], prefix=each, drop_first = True)
    data = pd.concat([data, dummies], axis = 1)

#target data is to be able to predict winning players
target_fields = ['won']
X, y = data.drop(target_fields, axis = 1), data[target_fields]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42, shuffle = True )

In [136]:
# create model

n_cols = X_train.shape[1]
model = Sequential()
model.add(Dropout(0.25, seed = 42, input_shape = (n_cols, )))

model = tf.keras.models.Sequential()
model.add(layers.Dense(12, activation='relu'))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [137]:
# Compile model
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 3)
history = model.fit(X_train, y_train, epochs = 20, validation_split = 0.25, batch_size = 128, callbacks = [early_stopping])

ValueError: Please provide as model inputs either a single array or a list of arrays. You passed: x=            Location                                    Tournament  \
15397          Sofia                       Garanti Koza Sofia Open   
9568      Cincinnati    Western & Southern Financial Group Masters   
15045         Vienna                               Erste Bank Open   
11570         London                                     Wimbledon   
3396      Casablanca                          Grand Prix Hassan II   
7781        Auckland                                 Heineken Open   
520        Marseille                                       Open 13   
12334        Beijing                                    China Open   
131         Auckland                                 Heineken Open   
6087        Belgrade                                   Serbia Open   
16978       Montreal                                Rogers Masters   
14600  Winston-Salem  Winston-Salem Open at Wake Forest Univ

In [138]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc = 'lower right')
plt.show()

NameError: name 'history' is not defined

In [128]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

NameError: name 'history' is not defined

In [129]:
print(model.evaluate(X_test, y_test))
mod_wts = model.get_weights()
model.summary()

ValueError: Error when checking input: expected dense_15_input to have shape (2908,) but got array with shape (138,)